## Neural Graph Collaborative Filtering

Here I intend to describe the algorithm presented in [Wang Xiang et al. Neural Graph Collaborative Filtering](https://arxiv.org/pdf/1905.08108.pdf) and my implementation using pytorch. 

I will go step by step (where *"step"* is mostly defined by my understanding) with snippets in tensorflow and their corresponding "translation" into pytorch.

Here we go

In [1]:
import numpy as np
import os
import sys
import argparse
import random as rd
import scipy.sparse as sp

import tensorflow as tf

import torch
from torch import nn
import torch.nn.functional as F

from utils.dataset import Data
from utils.metrics import *
from utils.parser import parse_args

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


I will be *"imperatively"* comparing `tf` and `pytorch`, and for that we need `tf` eager execution enabled

In [2]:
tf.enable_eager_execution()

We will be using our toy-data example so this notebook can initially run in any laptop

In [3]:
# 1000 users and 2000 items
data_path = "Data/toy_data/"
batch_size = 32
# for the toy dataset I did not create a validation set
data_generator = Data(data_path, batch_size, val=False)
_, _, mean_adj = data_generator.get_adj_mat()
adjacency_matrix = mean_adj + sp.eye(mean_adj.shape[0])
n_users = data_generator.n_users
n_items = data_generator.n_items

n_users=1000, n_items=2000
n_interactions=30780
n_train=24228, n_test=6552, sparsity=0.01539
already load adj matrix (3000, 3000) 0.021552085876464844


In [4]:
# model parameters
emb_size = 12
layers = [12, 6]
n_layers = len(layers)
node_dropout = 0.1
mess_dropout = [0.1]*len(layers)
regularization = 1e-5
lr = 0.01
n_fold = 10

Let's initialise weights. They (Wang Xiang et al) do it internally in their `NGCF` class, considering already the use of pre-trained weights. In my case, I will simply initialise all weights as if there were not pretrained and I will deal with pretrained weights outside the class

## TF

Their code:

In [5]:
pretrain_data = None
def _init_weights_tf():
    all_weights = dict()

    initializer = tf.contrib.layers.xavier_initializer()

    if pretrain_data is None:
        all_weights['user_embedding'] = tf.Variable(initializer([n_users, emb_size]), name='user_embedding')
        all_weights['item_embedding'] = tf.Variable(initializer([n_items, emb_size]), name='item_embedding')
        print('using xavier initialization')
    else:
        all_weights['user_embedding'] = tf.Variable(initial_value=pretrain_data['user_embed'], trainable=True,
                                                    name='user_embedding', dtype=tf.float32)
        all_weights['item_embedding'] = tf.Variable(initial_value=pretrain_data['item_embed'], trainable=True,
                                                    name='item_embedding', dtype=tf.float32)
        print('using pretrained initialization')

    weight_size_list = [emb_size] + layers
    for k in range(n_layers):
        # k = 0 are the embeddings
        all_weights['W_gc_%d' %k] = tf.Variable(
            initializer([weight_size_list[k], weight_size_list[k+1]]), name='W_gc_%d' % k)
        all_weights['b_gc_%d' %k] = tf.Variable(
            initializer([1, weight_size_list[k+1]]), name='b_gc_%d' % k)

        all_weights['W_bi_%d' % k] = tf.Variable(
            initializer([weight_size_list[k], weight_size_list[k + 1]]), name='W_bi_%d' % k)
        all_weights['b_bi_%d' % k] = tf.Variable(
            initializer([1, weight_size_list[k+1]]), name='b_bi_%d' % k)
    return all_weights

In [6]:
weights = _init_weights_tf()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

using xavier initialization


In [7]:
len(weights)

10

In [8]:
weights.keys()

dict_keys(['user_embedding', 'item_embedding', 'W_gc_0', 'b_gc_0', 'W_bi_0', 'b_bi_0', 'W_gc_1', 'b_gc_1', 'W_bi_1', 'b_bi_1'])

In [9]:
print(weights['user_embedding'].shape)
print(weights['item_embedding'].shape)
print(weights['W_gc_0'].shape)
print(weights['W_gc_1'].shape)

(1000, 12)
(2000, 12)
(12, 12)
(12, 6)


## Pytorch

As I mentioned, I will simply initialise the weights as with no pretrained and I will deal with pretrained weights outside the model class (see `run_bpr.py`)

In pytorch I simply do this:

In [11]:
# Do not run. This will go inside the model class
def _init_weights(self):
    for m in self.modules():
        if isinstance(m, nn.Embedding):
            nn.init.xavier_uniform_(m.weight)
        elif isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            nn.init.constant_(m.bias, 0.)

let's now focus on the 4 helpers that will be needed to create the model

## TF

Convert sparse matrix to sparse tensor

In [12]:
def _convert_sp_mat_to_sp_tensor_tf(X):
    coo = X.tocoo().astype(np.float32)
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

Node Dropout a for sparse tensor (see below, I'd call this edge dropout)

In [13]:
def _dropout_sparse(X, keep_prob, n_nonzero_elems):
    """
    Dropout for sparse tensors.
    """
    noise_shape = [n_nonzero_elems]
    random_tensor = keep_prob
    random_tensor += tf.random_uniform(noise_shape)
    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
    pre_out = tf.sparse_retain(X, dropout_mask)
    
    return pre_out * tf.math.divide(1., keep_prob)

Split the large sparse adjancecy matrix in n folds, with/without node dropout

In [14]:
def _split_A_hat_tf(X):
    "split the Adjancency matrix so is tractable"
    A_fold_hat = []

    fold_len = (n_users + n_items) // n_fold
    for i_fold in range(n_fold):
        start = i_fold * fold_len
        if i_fold == n_fold -1:
            end = n_users + n_items
        else:
            end = (i_fold + 1) * fold_len

        A_fold_hat.append(_convert_sp_mat_to_sp_tensor_tf(X[start:end]))
    return A_fold_hat

def _split_A_hat_node_dropout_tf(X):
    A_fold_hat = []

    fold_len = (n_users + n_items) // n_fold
    for i_fold in range(n_fold):
        start = i_fold * fold_len
        if i_fold == n_fold -1:
            end = n_users + n_items
        else:
            end = (i_fold + 1) * fold_len

        temp = _convert_sp_mat_to_sp_tensor_tf(X[start:end])
        n_nonzero_temp = X[start:end].count_nonzero()
        A_fold_hat.append(_dropout_sparse(temp, 1 - node_dropout, n_nonzero_temp))

    return A_fold_hat

Before moving into the code, I would like stop one second at their method `_dropout_sparse`. This method is intended to drop out nodes, and all their connections. However, I do not think the code in their original repo (the same as above) does that. I think their code drops edges. Let's see. After running: 

    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
   
they have a boolean tensor indicating **the locations** to keep (see [here](https://www.tensorflow.org/api_docs/python/tf/sparse/retain)). Therefore, when applying `dropout_mask` through: 

    pre_out = tf.sparse_retain(X, dropout_mask)
    
they are removing specific locations in the adjancency matrix, i.e. edges, not a node. Dropping a node (with all its connections) would imply dropping an entire row of the adjancency matrix, not only locations.

Anyway, let's see how those functions work

In [15]:
# Remember
adjacency_matrix

<3000x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 51456 stored elements in Compressed Sparse Row format>

In [16]:
adjacency_tensor = _convert_sp_mat_to_sp_tensor_tf(adjacency_matrix) 

In [17]:
print(adjacency_tensor.shape)

(3000, 3000)


In [18]:
A_fold_hat = _split_A_hat_node_dropout_tf(adjacency_matrix)

In [19]:
A_fold_hat[0].shape

TensorShape([Dimension(300), Dimension(3000)])

## Pytorch

The following functions are the adaptation to `Pytorch` of the functions above. In addition, I have added one additional form of dropout. Let's go, first, convert a sparse matrix into a sparse tensor using Pytorch `sparse` API. 

In [20]:
def convert_sp_mat_to_sp_tensor(X):
    coo = X.tocoo().astype(np.float32)
    i = torch.LongTensor(np.mat([coo.row, coo.col]))
    v = torch.FloatTensor(coo.data)
    return torch.sparse.FloatTensor(i, v, coo.shape)

In [21]:
adjacency_tensor = convert_sp_mat_to_sp_tensor(adjacency_matrix)

In [22]:
adjacency_tensor

tensor(indices=tensor([[   0, 1056, 1108,  ...,  969,  981, 2999],
                       [   0,    0,    0,  ..., 2999, 2999, 2999]]),
       values=tensor([1.0000, 0.0714, 0.0625,  ..., 0.0357, 0.0833, 1.0000]),
       size=(3000, 3000), nnz=51456, layout=torch.sparse_coo)

If we remember, before I said that I believe what the authors of the original paper (and code) refer as "node dropout" is in reality edge dropout. Dropping a node (i.e. all its connections) in a the graph involves dropping an entire row from the adjancency matrix. With that in mind, I included two methods, `_edge_dropout_sparse` designed to reproduce the dropout in the original code and `_node_dropout_sparse` which will drop a row (i.e. node) in the adjancency matrix. Let's have a look

In [23]:
def _edge_dropout_sparse(self, X, keep_prob):

    random_tensor = keep_prob
    random_tensor += torch.FloatTensor(X._nnz()).uniform_()
    dropout_mask = random_tensor.floor()
    dropout_tensor = torch.sparse.FloatTensor(X.coalesce().indices(), dropout_mask, X.size())
    X_w_dropout = X.mul(dropout_tensor)

    return  X_w_dropout.mul(1./keep_prob)

def _node_dropout_sparse(self, X, keep_prob):

    random_array = keep_prob
    random_array += np.random.rand(X.size()[0])
    dropout_mask = np.floor(random_array)
    dropout_mask = np.tile(dropout_mask.reshape(-1,1), X.size()[1])
    dropout_tensor = self.convert_sp_mat_to_sp_tensor(sp.csr_matrix(dropout_mask))
    X_w_dropout = X.mul(dropout_tensor)

    return  X_w_dropout.mul(1./keep_prob)

the first one is fairly straightforward to read, since the process is nearly identical to the one in `_dropout_sparse`. The only difference is that `pytorch` does not have an equivalent to `tf.sparse_retain()`. However, that's ok, one can simply do an element-wise multiplication to set to zero `(1-keep_prob)%` of the  elements. Regarding to the second one, let's go line by line:

In [24]:
keep_prob = 1 - node_dropout
random_array = keep_prob
random_array += np.random.rand(adjacency_tensor.size()[0])

In [25]:
print(random_array.shape)
print(random_array[:10])

(3000,)
[1.80294764 1.85788471 1.6895335  1.45745756 1.71669362 1.69276678
 1.82779998 1.86452777 1.19759748 1.5501779 ]


Now we have an array of 3000 numbers with a 90% of them greater 1.

In [26]:
dropout_mask = np.floor(random_array)
dropout_mask = np.tile(dropout_mask.reshape(-1,1), adjacency_tensor.size()[1])

In [27]:
print(dropout_mask.shape)
print(dropout_mask[:5, :5])

(3000, 3000)
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


And that's it, we now have a matrix where 90% of the rows are all ones and 10% are all 0s. When element-wise multiply by `adjacency_tensor` it will drop a node and all its connections.

The next two methods are identical to those above for tensorflow with very minor adaptations

In [28]:
def _split_A_hat(X):
    A_fold_hat = []

    fold_len = (n_users + n_items) // n_fold
    for i_fold in range(n_fold):
        start = i_fold * fold_len
        if i_fold == n_fold -1:
            end = n_users + n_items
        else:
            end = (i_fold + 1) * fold_len

        A_fold_hat.append(convert_sp_mat_to_sp_tensor(X[start:end]))
    return A_fold_hat

def _split_A_hat_node_dropout(X):
    A_fold_hat = []

    fold_len = (n_users + n_items) // n_fold
    for i_fold in range(n_fold):
        start = i_fold * fold_len
        if i_fold == n_fold -1:
            end = n_users + n_items
        else:
            end = (i_fold + 1) * fold_len

        temp = convert_sp_mat_to_sp_tensor(X[start:end])
        A_fold_hat.append(_edge_dropout_sparse(temp, 1 - node_dropout))

    return A_fold_hat

## The Model

We can now jump into the design of the model. Let's first include here the relevant figures from their paper as well as the relevant mathematical expressions. 

The figures below and the mathematical expression are taken directly from their paper. Thefore, as I have mentioned a number of times, **all credit to the authors**.

In [29]:
from IPython.display import HTML, display
display(HTML("<table><tr><td><img src='figures/Figure1.png'></td><td><img src='figures/Figure2.png'></td></tr></table>"))

,


The Figure on the left is an illustration of the user-item interaction graph, and what the authors call the high-order connectivity. The node/user $u_{1}$ is the target user to provide recommendations for. We can see that at first order ($l$-hop with $l$=1), one would capture the information that the user interacted with items 1, 2 and 3. At 2nd order (2-hop), one would capture $u_{1} \leftarrow i_{2} \leftarrow u_{2}$ and $u_{1} \leftarrow i_{3} \leftarrow u_{3}$, and so on.

To the right we have the model scheme. In the author's words: *"we design a neural network method to propagate embeddings recursively on the graph. This is inspired by the recent developments of graph neural networks [...], which can be seen as constructing information flows in the embedding space. Specifically, we devise an embedding propagation layer, which refines a user’s (or an item’s) embedding by aggregating the embeddings of the interacted items (or users). By stacking multiple embedding propagation layers, we can enforce the embeddings to capture the collaborative signal in high-order connectivities."*

To be honest this is one of these cases where the math (and then the code) will help to understand what is going on. Formally, the model consists of two pieces: *message construction* and *message aggregation*.  

**Message Construction**: for a connected user, item, a message is defined as (their expression 3):


$$
\textbf{m}_{u \leftarrow i} = \frac{1}{\sqrt{|\mathcal{N}_{u}||\mathcal{N}_{i}}|}\Big( \textbf{W}_{1} e_i + \textbf{W}_{2}(e_i \odot e_u) \Big)
$$


where $\odot$ denotes element-wise multiplication. $1/{\sqrt{|\mathcal{N}_{u}||\mathcal{N}_{i}}|}$ is the graph Laplacion norm, where $\mathcal{N}_{u}$ and $\mathcal{N}_{i}$ are the first-hop neighbors of user u and item i. Note that this factor (decay factor between two connected nodes) is already accounted by in our Laplacian matrix by construction. 

Also remember here $e_i$ or $e_u$ are *not* the initial embeddings, but the aggregated embeddings, i.e. for user 1, $e_{i}$ would be the aggregated embeddings of all the items that that user interacted with. This will be simply achieved by multiplying the initial embeddings by the Laplacian matrix. We'll see later. 


As simple as that, the "messages" are constructed, now we need to aggregate them. 

**Message Aggregation**: simply ((their expression 4): 

$$
e^{(1)}_{u} = \text{LeakyRelu} \Big( \textbf{m}_{u \leftarrow u} + \sum_{i \in \mathcal{N}_{u}} \textbf{m}_{u \leftarrow i} \Big)
$$

And from here basically, one repeats the process through as many layers as you might want. If you want to know more about all the reasoning behind the formulation of the message construction and aggregation, please, go to the paper. Is easy to read and understand. For what I need for this notebook these two expressions would be enough.

## TF

In [30]:
# let's assume no node dropout
A_fold_hat = _split_A_hat_tf(adjacency_matrix)

# they call this ego embeddings because they relate to ego-networks (i.e. 1st order connections)
# Simply the concatenation over rows of user and item embeddings. Shape  (n_users+n_items, n_emb)
ego_embeddings = tf.concat([weights['user_embedding'], weights['item_embedding']], axis=0)

Then, per *"graph layer"*, we do:

In [31]:
temp_embed = []
for f in range(n_fold):
    temp_embed.append(tf.sparse_tensor_dense_matmul(A_fold_hat[f], ego_embeddings))
side_embeddings = tf.concat(temp_embed, 0)

`side_embeddings` contains, for a given user, the weighted sum of the item embeddings that a certain user interacted with (plus the embeddings of that user). Also, for a given item, contains the weighted sum of the user embeddings that an item "interacted" with (plus the embeddings of that item). The decay factors (or the weights in the weighted sum) are $1/{\sqrt{|\mathcal{N}_{u}||\mathcal{N}_{i}}|}$. More formally, `side_embeddings` is: 

$$
\text{side_embeddings} =  \frac{1}{\sqrt{|\mathcal{N}_{u}||\mathcal{N}_{i}}|}e_i
$$

We now move to the second term of their expression (3). The authors refer to the element-wise multiplication $e_i \odot e_u$ as `bi_embeddings`

In [32]:
bi_embeddings = tf.multiply(ego_embeddings, side_embeddings)

More formally:

$$
\text{bi_embeddings} = \frac{1}{\sqrt{|\mathcal{N}_{u}||\mathcal{N}_{i}}|} (e_i \odot e_u)
$$

With all that, the only thing left to do to calculate:

$$
e^{(1)}_{u} = \text{LeakyRelu} \Big( \textbf{m}_{u \leftarrow u} + \sum_{i \in \mathcal{N}_{u}} \textbf{m}_{u \leftarrow i} \Big)
$$

with $l=0$ (first layer) is

In [33]:
#let's assume we are in the first iteration of the loop through layers, so k=0
k=0 
sum_embeddings = tf.nn.leaky_relu(tf.matmul(side_embeddings, weights['W_gc_%d' % k]) + weights['b_gc_%d' % k])
bi_embeddings = tf.nn.leaky_relu(tf.matmul(bi_embeddings, weights['W_bi_%d' % k]) + weights['b_bi_%d' % k])

# redefine the ego embeddings so the information of the 1st order interactions is captured
ego_embeddings = sum_embeddings + bi_embeddings
ego_embeddings = tf.nn.dropout(ego_embeddings, keep_prob=1-mess_dropout[k])

# some normalization
norm_embeddings = tf.math.l2_normalize(ego_embeddings, axis=1)

# back to user and item dim
u_g_emb, i_g_emb = tf.split(norm_embeddings, [n_users, n_items], 0)

Instructions for updating:
Colocations handled automatically by placer.


In [37]:
u_g_emb.shape, i_g_emb.shape

(TensorShape([Dimension(1000), Dimension(12)]),
 TensorShape([Dimension(2000), Dimension(12)]))

## Pytorch

In [38]:
# we have not defined the model components yet in pytorch :) 
# Note that the embeddings could also be defined as tensors:

# embeddings_user = nn.Parameter(torch.rand(n_users, emb_dim))

# but I prefer to use modules, so they print out when you call 
# the model and is easier to access to them internally within 
# the model class. In the end is a a matter of taste
embeddings_user = nn.Embedding(n_users, emb_size)
embeddings_item = nn.Embedding(n_items, emb_size)
W1 = nn.ModuleList()
W2 = nn.ModuleList()

features = [emb_size] + layers
for i in range(1,len(features)):
        W1.append(nn.Linear(features[i-1],features[i]))
        W2.append(nn.Linear(features[i-1],features[i]))

In [39]:
embeddings_user

Embedding(1000, 12)

In [40]:
# let's assume no node dropout
A_fold_hat = _split_A_hat(adjacency_matrix)

# they call this ego embeddings because they relate to ego-networks (i.e. 1st order connections)
# Simply the concatenation over rows of user and item embeddings. Shape  (n_users+n_items, n_emb)
ego_embeddings = torch.cat([embeddings_user.weight, embeddings_item.weight], 0)

In [41]:
ego_embeddings.shape

torch.Size([3000, 12])

In [42]:
temp_embed = []
for f in range(n_fold):
    temp_embed.append(torch.sparse.mm(A_fold_hat[f], ego_embeddings))
weighted_sum_emb = torch.cat(temp_embed, 0)

In [43]:
# note that tf.matmul(side_embeddings, weights['W_gc_%d' % k]) + weights['b_gc_%d' % k]
# is just what a linear layer would do. So I implemented the first term of the summation (t1) as:
t1 = W1[k](weighted_sum_emb)

$$
\text{t1} =  \frac{1}{\sqrt{|\mathcal{N}_{u}||\mathcal{N}_{i}}|}e_i
$$

In [44]:
# in their paper they say that the element-wise multiplication makes the message dependent 
# on the affinity between ei and eu. So I call them affinity_emb and implemented the term 2 (t2) as:
affinity_emb = ego_embeddings.mul(weighted_sum_emb)
t2 = W2[k](affinity_emb)

$$
\text{t2} = \frac{1}{\sqrt{|\mathcal{N}_{u}||\mathcal{N}_{i}}|} (e_i \odot e_u)
$$

so $e^{(l=0)}_u$ is

In [45]:
# unlike tf, nn.Dropout takes p, prob of element to be zeroed.
ego_embeddings = nn.Dropout(mess_dropout[k])(F.leaky_relu(t1 + t2))

# normalization
norm_embeddings = F.normalize(ego_embeddings, p=2, dim=1)

# back to user and item dim
u_g_emb, i_g_emb= norm_embeddings.split([n_users, n_items], 0)

In [46]:
u_g_emb.shape, i_g_emb.shape

(torch.Size([1000, 12]), torch.Size([2000, 12]))

THE END